In [ ]:
# Bibliotecas
from pyspark.sql.types import *
import os

In [ ]:
# Criacao de tabelas
def create_table(table_name:str, has_input:bool, in_fld:str='/FileStore/tables/raw_file/delta'):
    if has_input:
        # Deleta a tabela caso exista
        spark.sql(f"""
        DROP TABLE IF EXISTS {table_name}""")
        # cria uma nova tabela com os dados delta tratados anteriormente
        spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {table_name}
        USING DELTA
        LOCATION '{in_fld}'
        """)
        # Por termos uma serie de arquivos pequenos, 'e necessario
        # rodar o optimize para reduzir arquivos pequenos em maiores
        spark.sql("""
        OPTIMIZE default.raw
        """)
    else:
        # Deleta a tabela caso exista
        spark.sql(f"""
        DROP TABLE IF EXISTS {table_name};
        """)
        # cria uma nova tabela com os dados delta tratados anteriormente
        spark.sql(f"""
        CREATE TABLE IF NOT EXISTS {table_name}
            (
            dateTimeReference timestamp,
            modifiedDate timestamp
            idRetailerSKU string,
            seller string,
            retailerProductCode string,
            retailerAverageRating float,
            retailerRatingCount int,
            retailerPrice float, 
            manufacturerPrice float, 
            priceVariation float,
            available boolean,
            unavailable boolean,
            notListed boolean,
            titleFlag boolean,
            titlePercentage float
            );
        """)


In [ ]:
try:
    # primeiramente, foi 'e aconselhado passar a estrutura de leitura do json
    # quando for realizar a leitura do arquivo pois ele ja trata no pyspark
    # possiveis problemas de tipagem ou inconsistencia de dados
    schema = StructType([
        StructField("assortment", ArrayType(
            StructType([
                StructField('dateTimeReference', TimestampType(), True),
                StructField('modifiedDate', TimestampType(), True),
                StructField('idRetailerSKU', StringType(), True),
                StructField('seller', StringType(), True),
                StructField('retailerProductCode', StringType(), True),
                StructField("retailerAverageRating", FloatType(), True),
                StructField("retailerRatingCount", IntegerType(), True),
                StructField('retailerPrice', FloatType(),True),
                StructField('manufacturerPrice', FloatType(),True),
                StructField('priceVariation', FloatType(),True),
                StructField('available', BooleanType(),True),
                StructField('unavailable', BooleanType(),True),
                StructField('notListed', BooleanType(),True),
                StructField('titleFlag', BooleanType(),True),
                StructField('titlePercentage', FloatType(),True),
            ])
        ))
    ])

    # chamamos a pasta de arquivos e inserimos todos os valores linha a linha em
    # um rdd com os dados
    df = spark.read.schema(schema).json( "/FileStore/tables/raw_file/data-eng-test/")
    # aqui 'e feita a conversao do tipo de arquivo para delta
    # se ja existem arquivos dentro da pasta delta
    if dbutils.fs.ls("/FileStore/tables/raw_file/delta_lake/"):
        dbutils.fs.rm("/FileStore/tables/raw_file/delta_lake",True)
    df.write.format('delta').mode('overwrite').save('/FileStore/tables/raw_file/delta_lake')
# tratamento da excecao de caso nao haja a pasta dos arquivos selecionados
except Exception as e:
    print(f'Houve um erro na extracao doas dados. Erro: {e}')
# mensagem de finalizacao do notebook
else: 
    # Cria tabela com os dados presentes dos arquivos delta gerados
    # no notebook anterior
    try:
        create_table(table_name='default.bronze', 
                    has_input=True, 
                    in_fld='/FileStore/tables/raw_file/delta_lake')
    except Exception as e:
        print("Nao foi possivel fazer a carga da tabela" + '\n' + f'Erro : {e}')
    else:
        print('Dados carregados com sucesso!')
            

In [ ]:
# Cria tabela com os dados presentes dos arquivos delta gerados
